In [10]:
import pandas as pd
from collections import deque

In [11]:
plays = pd.read_csv('../data/play_by_play.txt', delimiter = "\t")
lineup = pd.read_csv('../data/lineup.txt', delimiter = "\t").sort_values(['Game_id', 'Period', 'Team_id'], 
                                                                         ascending = [True, True, True])
plays = plays.sort_values(['Game_id', 'Period', 'PC_Time', 'WC_Time', 'Event_Num'], 
                 axis = 0, ascending = [True, True, False, True, True]).drop(['Team_id_type', 'Option2', 'Option3'], axis = 1)
game_ids = plays['Game_id'].unique().tolist()

#working with game 0
lineup0 = lineup[lineup.Game_id == game_ids[0]].reset_index()
df = plays[plays.Game_id == game_ids[0]]



# something with views if error in future

In [12]:
team1players = {}
team2players = {}

t1points = {}
t2points = {}

t1Active = []
t2Active = []
team1 = ''
team2 = ''
bigList1 = []
bigList2 = []
for i, row in lineup0.iterrows():
    if i < 10:
        if i < 5:
            team1players[row['Person_id']] = 0
            t1Active.append(row['Person_id'])
            team1 = row["Team_id"]
            t1points[row['Person_id']] = 0
        else:
            team2players[row['Person_id']] = 0
            t2points[row['Person_id']] = 0
            t2Active.append(row['Person_id'])
            team2 = row["Team_id"]
            
bigList1.append(t1Active)
bigList2.append(t2Active)
subsInFoul = deque()
flag = False

#print(df.loc[147:157])
for i, event in df.iterrows():
    if (i==146):
        continue
#    print(team1players)
#    print(team2players)
    print(t1Active)
    print(t2Active)
    print(subsInFoul)
    
    event_type = event['Event_Msg_Type']
    #1-made shot, 2-missed shot, 3-free throw, 4-rebound, 5-turnover, 6-foul, 7-violation, 8-substitution, 
    #9-Timeout, 10-Jump Ball, 11-Ejection, 12-Start Period, 13-End Period
    
    if event_type == 6:
        flag = True
        #is a free throw happening
        
    if event_type == 8: 
        person1 = event['Person1']
        person2 = event['Person2']
#        print(person1, person2, event['Team_id'])    
        if not flag: #if free throw is definitely not happening
        
            if person1 in team1players:
                t1Active.remove(person1)
                t1Active.append(person2)
                if person2 not in team1players:
                    team1players[person2] = 0 #add them to the dictionary
                    t1points[person2] = 0
            else:
                t2Active.remove(person1)
                t2Active.append(person2)
                if person2 not in team2players:
                    team2players[person2] = 0
                    t2points[person2] = 0
                    
        else: #free throw might be happening, add them to the queue
#            subsInFoul.append((person1, person2,event['Team_id']))

            subsInFoul.append( (person1, person2))
    
    if (flag == True) and (event_type == 1 or event_type == 2 or event_type == 4 or event_type == 5 or event_type == 10 
                           or event_type==7):
        #play went on, foul did not result in free throws
        flag = False

        
        while len(subsInFoul) != 0:
            
          
            sub = subsInFoul.popleft()
#            print(sub)
            if sub[0] in team1players:
                t1Active.remove(sub[0])
                t1Active.append(sub[1])
                if sub[1] not in team1players:
                    team1players[sub[1]] = 0
                    t1points[sub[1]] = 0
            else:
                t2Active.remove(sub[0])
                t2Active.append(sub[1])
                if sub[1] not in team2players:
                    team2players[sub[1]] = 0
                    t2points[sub[1]] = 0
        
                    
    #still have free throws, violation
    bigList1.append(t1Active)
    bigList2.append(t2Active)
    
    if (flag == False) and (event_type==1 or event_type==2 or event_type == 4 or event_type == 5 or event_type == 10 
                           or event_type==7):
        if (event_type==1): #Made shot
            person1 = event['Person1']
            person2 = event['Person2']
            if (person1 in team1players):
                for i in t1Active:
                    team1players[i] += event['Option1']

                for i in t2Active:
                    team2players[i] -= event['Option1']
                    
                t1points[person1] += event['Option1']
            else:
                for i in t1Active:
                    team1players[i] -= event['Option1']
                for i in t2Active:
                    team2players[i] += event['Option1']          
                t2points[person1] += event['Option1']
    #covered 1,2,4,5,6,7,8, 10
    #9, 11,12,13 are all useless
    if (event_type==3):
        person1 = event['Person1']
        person2 = event['Person2']
        if (person1 in team1players):
            for i in t1Active:
                team1players[i] += event['Option1']
            for i in t2Active:
                team2players[i] -= event['Option1']
            t1points[person1] += event['Option1']
                
        else:
            for i in t1Active:
                team1players[i] -= event['Option1']
            for i in t2Active:
                team2players[i] += event['Option1']
            t2points[person1] += event['Option1']
    print(t1points)
    print(t2points)
    print("\n")



['881f83d2dee3f18c7d1751659406144e', 'cec898a1d355dbfbad8c760615fde1af', 'a99f44bbff39e352191a870e17f04537', '89706b99ddd00dc05d37ef5cafc04276', '2b313e2bcef0268bc8e9415132ba9997']
['27ea17a8685c4919f157e83fe9cb2d9e', '57bbd7e30bc694aeee9ee40c583e6811', '33963fe856a1523ff46438ba07d1d99f', 'c00264c3114d23bac482e9de50fb7d28', '307beab25b1021a548b4a47550bc4b25']
deque([])
{'881f83d2dee3f18c7d1751659406144e': 0, 'cec898a1d355dbfbad8c760615fde1af': 0, 'a99f44bbff39e352191a870e17f04537': 0, '89706b99ddd00dc05d37ef5cafc04276': 0, '2b313e2bcef0268bc8e9415132ba9997': 0}
{'27ea17a8685c4919f157e83fe9cb2d9e': 0, '57bbd7e30bc694aeee9ee40c583e6811': 0, '33963fe856a1523ff46438ba07d1d99f': 0, 'c00264c3114d23bac482e9de50fb7d28': 0, '307beab25b1021a548b4a47550bc4b25': 0}


['881f83d2dee3f18c7d1751659406144e', 'cec898a1d355dbfbad8c760615fde1af', 'a99f44bbff39e352191a870e17f04537', '89706b99ddd00dc05d37ef5cafc04276', '2b313e2bcef0268bc8e9415132ba9997']
['27ea17a8685c4919f157e83fe9cb2d9e', '57bbd7e30bc694a

ValueError: list.remove(x): x not in list